In [1]:
# import packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import haversine as hs
import gpxpy
import gpxpy.gpx
import statsmodels.api as sm
from datetime import datetime
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

In [2]:
# define functions to be or not to be used
def gpx_to_df(gpx_dat):
    '''
    converts a gpx file to a csv. needs headers for latitude, longitude, and elevation to be pre-cleaned
    '''
    import pandas as pd
    coord_lst = []

    for point in route_gpx.tracks[1].segments[0].points:
        coord_lst.append({
            'latitude': point.latitude,
            'longitude': point.longitude,
            'elevation': point.elevation
                })
    return pd.DataFrame(coord_lst)

def haversine_distance(lat1, lon1, lat2, lon2) -> float:
    '''
    order = lat1, lon1, lat2, lon2
    '''
    import haversine as hs
    distance = hs.haversine(
        point1=(lat1, lon1),
        point2=(lat2, lon2),
        unit=hs.Unit.METERS
    )
    return np.round(distance, 4)

def seq_vis(df, colname, col = '#101010'):
    to_visualize = df[str(colname)]
    plt.figure(figsize = (15, 10))
    plt.title(f'{colname.title()} on the route', size=22)
    plt.xlabel('Data point', size=14)
    plt.ylabel(f'{colname}', size=14)
    plt.plot(np.arange(len(to_visualize)), to_visualize, lw=2, color=col)

In [3]:
easter_csv = pd.read_csv('/Users/chriscraig/Desktop/projects/trails/My Map.csv')

In [4]:
# wrangle
easter_csv = easter_csv.rename(columns = {'Latitude':'latitude', 'Longitude':'longitude', 'Elevation (m)':'elevation', 'Timestamp UTC (ms)':'timestamp'})


#distance
hav_distance = [np.nan]
lat = easter_csv.columns.get_loc('latitude')
lon = easter_csv.columns.get_loc('longitude')
for i in range(easter_csv.shape[0]):  
    if i == 0:
        pass
    else:
        dist = haversine_distance(easter_csv.iloc[i-1, lat], easter_csv.iloc[i-1, lon], easter_csv.iloc[i, lat], easter_csv.iloc[i, lon])
        hav_distance.append(dist)
easter_csv['distance'] = hav_distance

easter_csv['cum_dist'] = [np.nansum(easter_csv['distance'][0:i]) for i in range(easter_csv.shape[0])]


pct_per_point = .02

#smooth elevation
lowess = sm.nonparametric.lowess(endog = easter_csv['elevation'], exog = easter_csv['cum_dist'], frac= pct_per_point)
easter_csv['elevation_smooth'] = lowess[:,1]
easter_csv['elevation_smooth_diff'] = easter_csv['elevation_smooth'].diff()
gradient = [np.nan]

for i, row in easter_csv.iterrows():
    if i == 0:
        pass
    elif row['distance'] == 0:
        gradient.append(0)
    else:
        gradient.append(round(row['elevation_smooth_diff']/row['distance'] * 100, 2))

#elevation dif
easter_csv['elevation_diff'] = easter_csv['elevation_smooth'].diff()

#gradient
gradient = [np.nan]
for i, row in easter_csv.iterrows():
    if i == 0:
        pass
    elif row['distance'] == 0:
        gradient.append(0)
    else:
        gradient.append(round(row['elevation_diff']/row['distance'] * 100, 2))

for i in range(len(gradient)):
    if abs(gradient[i]) > 30:
        gradient[i] = np.nan

easter_csv['gradient'] = gradient




In [5]:
#time

easter_csv.timestamp = easter_csv.timestamp/1000
easter_csv['date'] = [datetime.fromtimestamp(easter_csv.timestamp[i], ) for i in range(len(easter_csv.timestamp))]

easter_csv['time_diff'] = easter_csv.date.diff()
easter_csv['time_diff'] = [row.total_seconds() for row in easter_csv['time_diff']]

#speed
easter_csv['speed'] = np.divide(easter_csv.distance, easter_csv.time_diff, out=np.zeros_like(easter_csv.distance),
                                where=(easter_csv.time_diff > 0 &(easter_csv.distance>10)))
# meters/s to miles/h
easter_csv['speed'] = easter_csv['speed']*2.23694

In [6]:
# visuals

In [10]:
midpoint_lat = (np.max(easter_csv.latitude) + np.min(easter_csv.latitude))/2
midpoint_lon = (np.max(easter_csv.longitude) + np.min(easter_csv.longitude))/2

41.561137

In [12]:
# map
route_map = folium.Map(
        location=[midpoint_lat, midpoint_lon],
    zoom_start=12,
    tiles='CartoDBPositron', #CartoDBDark_Matter, CartoDBPositron
    width=1024,
    height=600
)

coordinates = [tuple(x) for x in easter_csv[['latitude', 'longitude']].to_numpy()]
folium.PolyLine(coordinates, weight=4).add_to(route_map)

display(route_map)